In [36]:
import PyPDF2
import re

file_path = 'harrypotter.txt'

# Open the PDF file in binary mode
with open(file_path, "r",encoding="utf-8") as f:
    raw_text = f.read()
    

In [37]:
print("Total number of characters: ", len(raw_text))
print(raw_text[:109])  # Print the first 99 characters

Total number of characters:  6290999
CONTENTS
Harry Potter and the Sorcerer's Stone
Harry Potter and the Chamber of Secrets
Harry Potter and the P


In [38]:
import re
# test_text = "Hello world, this is-- a,test."
preprocessed = re.split(r'([,.:;?_!"()\']|-|--|\s)',raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:40])

['CONTENTS', 'Harry', 'Potter', 'and', 'the', 'Sorcerer', "'", 's', 'Stone', 'Harry', 'Potter', 'and', 'the', 'Chamber', 'of', 'Secrets', 'Harry', 'Potter', 'and', 'the', 'Prisoner', 'of', 'Azkaban', 'Harry', 'Potter', 'and', 'the', 'Goblet', 'of', 'Fire', 'Harry', 'Potter', 'and', 'the', 'Order', 'of', 'the', 'Phoenix', 'Harry', 'Potter']


In [39]:
print(len(preprocessed))

1448745


# Now we create token IDS

In [40]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(all_words[:40])
print(vocab_size)

['!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '10', '12', '125', '1289', '1296', '12th', '13', '1473', '1492', '15', '150', '160', '1612', '1612–1697', '1637', '1689', '17', '170', '1709', '1722', '1722–1741', '1741–1768', '1749', '1792', '18', '1875', '1890', '1909', '1945']
25861


In [41]:
vocab = {token:integer for integer,token  in enumerate(all_words)}

In [42]:
for i ,item in enumerate (vocab.items()):
    print(item)
    if (i>=5):
        break

('!', 0)
('"', 1)
('&', 2)
("'", 3)
('(', 4)
(')', 5)


In [43]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()       
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [44]:
tokenizer = SimpleTokenizerV1(vocab)

text = """  He was in a very good mood until lunchtime, when he
            thought he'd stretch his legs and walk across the road to buy himself a bun
            from the bakery."""
ids = tokenizer.encode(text)
print(ids)

[2728, 24911, 14799, 6594, 24648, 13617, 16820, 24412, 16171, 6, 25085, 14120, 23293, 14120, 3, 10389, 22431, 14330, 15825, 7065, 24825, 6740, 23209, 20167, 23470, 8754, 14309, 6594, 8672, 13180, 23209, 7580, 8]


In [45]:
tokenizer.decode(ids)

"He was in a very good mood until lunchtime, when he thought he' d stretch his legs and walk across the road to buy himself a bun from the bakery."

In [49]:
# text = "Babuji, do you like tea?"
# print(tokenizer.encode(text))

# ADDING SPECIAL CONTEXT TOKENS

In [50]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [51]:
len(vocab.items())

25863

In [35]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('“‘the', 27662)
('“‘—', 27663)
('…', 27664)
('<|endoftext|>', 27665)
('<|unk|>', 27666)


In [52]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [53]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [54]:
tokenizer.encode(text)

[2760,
 6,
 11259,
 25521,
 15923,
 23058,
 86,
 25861,
 2994,
 23209,
 22680,
 25862,
 17441,
 23209,
 17849,
 8]

In [55]:
tokenizer.decode(tokenizer.encode(text))

'Hello, do you like tea? <|endoftext|> In the sunlit <|unk|> of the palace.'

# BYTE PAIR ENCODING (BPE)

In [56]:
! pip3 install tiktoken

  Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
   ---------------------------------------- 0.0/894.0 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/894.0 kB ? eta -:--:--
   ----------------------- ---------------- 524.3/894.0 kB 1.9 MB/s eta 0:00:01
   ----------------------------------- ---- 786.4/894.0 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 894.0/894.0 kB 1.2 MB/s eta 0:00:00
Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl (274 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)

   ---------------------------------------- 0/7 [urllib3]
   ---------------------------------------- 0/7 [urllib3]
   ---------------------------------------- 0/7 [urllib3]
   ----- ---------------------------------- 1/7 [regex]
   ----------- ---------------------------- 2/7 [idna]
   ----------------- ---------------------- 3/7 [charset-normalizer]
   

In [57]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [58]:
tokenizer = tiktoken.get_encoding("gpt2")

In [59]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [60]:
integers = tokenizer.encode("Akwirw ier")
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


# DATA SAMPLING WITH SLIDING WINDOW